In [ ]:
%%capture
! pip install bespokelabs-curator

In [ ]:
import json
import pandas as pd
from bespokelabs import curator
from tqdm import tqdm
import pprint
from google import genai
llm = curator.LLM(model_name="gpt-4o", backend_params={"max_requests_per_minute": 200, "api_key": ''})

[05/06/25 02:01:29] INFO     Requesting text output from gpt-4o, using OpenAI backend                ]8;id=695582;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/_factory.py\_factory.py]8;;\:]8;id=992098;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/_factory.py#60\60]8;;\

In [ ]:
strategies = {
    'build_rapport' : "**Build Rapport:** Establish a respectful and understanding relationship with the Believer (e.g., to ensure the conversation is seen as a friendly exchange rather than a confrontation; demonstrating understanding and empathy towards the individuals beliefs without judgment).",
    'critical_thinking' : "**Critical Thinking:** Encourage the Believer to question and analyze the logic, evidence, and sources behind their beliefs, promoting a more analytical and reflective approach to information.",
    'alternative_explanations': "**Alternative Explanations: Provide plausible, evidence-based alternative perspectives or explanations for events or phenomena that are attributed to conspiracy theories.",
    'harm': "**Harm: Discuss the personal or societal harms of the conspiracy beliefs.",
    'stories':"**Stories/Examples:** Share stories, anecdotes, or real-world examples.",
    'encourage_empathy':"**Encourage Empathy:** Help the Believer consider the impact of conspiracy beliefs on others, fostering empathy and a broader perspective.",
    'socratic':"**Socratic Questioning:** Employ a questioning approach that leads the Believer to reflect on and examine the validity of their beliefs.",
    'conflicting_evidence':"**Conflicting Evidence:** Introduce facts or data that directly contradict claims made by the conspiracy theory or the Believer.",
    'common_ground':"**Common Ground/Shared Reality:** Identify and build on beliefs or values that the Debunker shares with the Believer.",
    'psychological_needs':"**Psychological Needs:** Recognize and address the emotional aspects or psychological needs that may be underlying the Believers attraction to conspiracy theories, such as a desire for control or understanding.",
    'inconsistencies':"**Inconsistencies/Logical Fallacies:** Identify and discuss logical inconsistencies or fallacies in the conspiracy theorys arguments."
}


PROMPT = """
  You are about to be shown the text of a written conversation about fake news. Your job is to process the conversation and return a classification of the nature of consultant's arguments. Particularly, you will determine whether the given arguments use the following persuasion strategy:
  {strategy}
  ** Response Scale **
  Use the following response scale for each rating:
  1 : Strategy not used.
  2 : Strategy used rarely, in a limited fashion.
  3 : Strategy used repeatedly or with clear emphasis.
  4 : Strategy used extensively and/or centrally throughout the response.”

  No need for explanations. Only analyze the Consultant's argument across all the rounds. Output only the correct response value.
  *{statement}*
 """


PROMPT_Reasoning = """
You are given a debate transcript and the final judge's reasoning. Your task is to process the transcript and identify if any of the given strategies persuaded the judge to make a decision.
  **Critical Thinking:** Encourage the Believer to question and analyze the logic, evidence, and sources behind their beliefs, promoting a more analytical and reflective approach to information.
  **Alternative Explanations: Provide plausible, evidence-based alternative perspectives or explanations for events or phenomena that are attributed to conspiracy theories.
  **Harm: Discuss the personal or societal harms of the conspiracy beliefs.
  **Stories/Examples:** Share stories, anecdotes, or real-world examples.
  **Conflicting Evidence:** Introduce facts or data that directly contradict claims made by the conspiracy theory or the Believer.
  **Inconsistencies/Logical Fallacies:** Identify and discuss logical inconsistencies or fallacies in the conspiracy theorys arguments.

Please give your answer in a brief statement.

Debate Transcript:
{debate}


Judge's Reasoning:
{reasoning}
"""

In [ ]:
#read data
filename = "/content/drive/MyDrive/Final-transcript-debate-consultancy/Human-debate/debate_all_negative_belief_data.json"
file = open(filename)
data = json.load(file)

In [ ]:
#read debate script
veracity = []
arguments = []
humans = []
answer_defending = []
sessions = []
topic = []
reasoning = []

for human, human_dict in data.items():
        for session in human_dict:
          humans.append(human)
          sessions.append(session['session_id'])
          topic.append(session['debate_topic'])
          full_argument = ''
          reasoning.append(session['reasoning'])
          for round in eval(session['rounds']):
            # print(round.keys())

            veracity.append(round['ground_truth_veracity'])
            answer_defending.append(round['debater_positions']['second_debater']['defending'])
            # rounds.append(round['round_number'])
            full_argument += f"Round {round['round_number']} \n\n Debater A: {round['debater_a_response']} \n\n Debater B: {round['debater_b_response']} \n\n Judge's Question: {round['judge_feedback']} \n\n"
          arguments.append(full_argument)


In [ ]:
#read consultant script
veracity = []
arguments = []
humans = []
answer_defending = []
sessions = []
topic = []

for human, human_dict in data.items():

        for session in human_dict:
          # print(human_dict[0].keys())
          humans.append(human)
          sessions.append(session['session_id'])
          veracity.append(session['veracity'])
          topic.append(session['debate_topic'])
          # print(session['debate_topic'])
          # print('_'*50)
          answer_defending.append(session['answer_defending'])
          # print(eval(human_dict[0]['rounds'])
          full_argument = ''
          for round in eval(session['rounds']):
            # print(round['round_number'])
            # print(round['judge_feedback'])
            # print(round['round_number'])
            # full_argument += f"Round {round['round_number']} : {round['judge_feedback']}"
            # print(round.keys())
            # rounds.append(round['round_number'])
            full_argument += f"Round {round['round_number']} \n\n Consultant's Argument: {round['consultant_response']} \n\n Judge's Question: {round['judge_feedback']} \n\n"



          arguments.append(full_argument)


In [ ]:
df = pd.DataFrame(list(zip(humans, sessions, arguments, topic, answer_defending, veracity, reasoning)),
    columns=['humans', 'sessions', 'arguments', 'debate_topic', 'answer_defending', 'veracity', 'reasoning'])

In [ ]:
df['arguments'][10]

"Round 1 \n\n Consultant's Argument: The claim that pregnant women with coronavirus are at an increased risk of severe illness is not conclusively supported by all available data. According to the KFF article, <v_evidence>initial studies from China suggest similar clinical characteristics and severity between pregnant and non-pregnant women</v_evidence> <url>https://www.kff.org/coronavirus-covid-19/issue-brief/novel-coronavirus-covid-19-special-considerations-for-pregnant-women/</url>. This indicates that the perceived increased risk may not be as significant as some reports suggest. Furthermore, the article highlights the limited data available, underscoring the need for more comprehensive research to fully understand the impact of COVID-19 on pregnant women. Without definitive evidence, it is premature to assert that pregnant women are at a higher risk of severe illness from COVID-19 compared to their non-pregnant counterparts. \n\n Judge's Question: Debater A provides clear CDC data

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   humans            218 non-null    object
 1   sessions          218 non-null    object
 2   arguments         218 non-null    object
 3   debate_topic      218 non-null    object
 4   answer_defending  218 non-null    object
 5   veracity          218 non-null    bool  
 6   reasoning         218 non-null    object
dtypes: bool(1), object(6)
memory usage: 10.6+ KB


In [ ]:
def format_prompt(text, strategy):
    return PROMPT.format(statement=text, strategy=strategy)

In [ ]:
! export CURATOR_DISABLE_CACHE=true

In [ ]:
for item, strategy in strategies.items():
  df['formatted_prompt'] = df.apply(lambda x: format_prompt(x['arguments'], strategy), axis=1)
  output = []
  for i in tqdm(range(len(df))):
    output.append(client.models.generate_content(model="gemini-2.0-flash", contents=df['formatted_prompt'][i]).text)
    # print(output)
  df[f"{item}_gemini"] = output
  responses = llm(df['formatted_prompt'][:50].to_list())


[05/06/25 02:08:31] INFO     Running OpenAIOnlineRequestProcessor completions with    ]8;id=29784;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=241286;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py#131\131]8;;\
                             model: gpt-4o                                                                         

[05/06/25 02:08:32] INFO     Using cached requests. If you want to regenerate the     ]8;id=463939;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=467354;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py#212\212]8;;\
                             dataset, disable or delete the cache.                                                 
                              See                                                                                  
                             https://docs.bespokelabs.ai/bespoke-curator/tutorials/au                              
                             tomatic-recovery-and-caching#disable-caching for more                                 
                             information.                                                                          

                    INFO     Manually set max_requests_per_minute to 200       ]8;id=253503;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py\base_online_request_processor.py]8;;\:]8;id=870160;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py#191\191]8;;\

                    INFO     Automatically set max_tokens_per_minute to 450000 ]8;id=669615;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py\base_online_request_processor.py]8;;\:]8;id=326177;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py#213\213]8;;\

                    INFO     Resuming progress by reading existing file:              ]8;id=3577;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=727241;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py#577\577]8;;\
                             /root/.cache/curator/ff2daddfe0a119b3/responses_0.jsonl                               

...

                    INFO     Status tracker: Tasks - Started: 27, In Progress: ]8;id=304238;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py\base_online_request_processor.py]8;;\:]8;id=473367;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py#457\457]8;;\
                             0, Succeeded: 27, Failed: 0, Already Completed:                                       
                             23                                                                                    
                             Errors - API: 0, Rate Limit: 0, Other: 0, Total:                                      
                             0                                                                                     

                    INFO     Read 50 responses.                                       ]8;id=521684;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=751195;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py#442\442]8;;\

                    INFO     Finalizing writer                                        ]8;id=589171;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=272903;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py#451\451]8;;\

                    INFO     Creating a file with all failed requests                 ]8;id=740037;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=765568;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py#460\460]8;;\

                    INFO     Created file with failed requests at                     ]8;id=403715;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=205863;file:///usr/local/lib/python3.11/dist-packages/bespokelabs/curator/request_processor/base_request_processor.py#488\488]8;;\
                             /root/.cache/curator/ff2daddfe0a119b3/failed_requests.js                              
                             onl                                                                                   

In [ ]:
#get the int number from gemini
df['critical_thinking_gemini'] = df['critical_thinking_gemini'].str.strip().astype(int)
df['alternative_explanations_gemini'] = df['alternative_explanations_gemini'].str.strip().astype(int)
df['harm_gemini'] = df['harm_gemini'].str.strip().astype(int)
df['stories_gemini'] = df['stories_gemini'].str.strip().astype(int)
df['conflicting_evidence_gemini']= df['conflicting_evidence_gemini'].str.strip().astype(int)
df['inconsistencies_gemini'] = df['inconsistencies_gemini'].str.strip().astype(int)


#extract the int number from gpt4o
df['critical_thinking_gpt4o'] = df['critical_thinking_gpt4o'].astype(str).str.extract(r'(-?\d+\.?\d*)').astype(float).astype(int)
df['alternative_explanations_gpt4o'] = df['alternative_explanations_gpt4o'].str.extract(r'(-?\d+\.?\d*)').astype(float).astype(int)
df['harm_gpt4o'] = df['harm_gpt4o'].astype(str).str.extract(r'(-?\d+\.?\d*)').astype(float).astype(int)
df['stories_gpt4o'] = df['stories_gpt4o'].astype(str).str.extract(r'(-?\d+\.?\d*)').astype(float).astype(int)
df['conflicting_evidence_gpt4o'] = df['conflicting_evidence_gpt4o'].str.extract(r'(-?\d+\.?\d*)').astype(float).astype(int)
df['inconsistencies_gpt4o'] = df['inconsistencies_gpt4o'].str.extract(r'(-?\d+\.?\d*)').astype(float).astype(int)


#get the average score
df['critical_thinking'] = (df['critical_thinking_gemini'] + df['critical_thinking_gpt4o'])/2
df['harm'] = (df['harm_gemini'] + df['harm_gpt4o'])/2
df['alternative_explanations'] = (df['alternative_explanations_gemini'] + df['alternative_explanations_gpt4o'])/2
df['stories'] = (df['stories_gemini'] + df['stories_gpt4o'])/2
df['inconsistencies'] = (df['inconsistencies_gemini'] + df['inconsistencies_gpt4o'])/2
df['conflicting_evidence'] = (df['conflicting_evidence_gemini'] + df['conflicting_evidence_gpt4o'])/2

In [ ]:
df['answer_defending'] = df['answer_defending'].astype(bool)

In [ ]:
df.to_csv("/content/drive/MyDrive/Final-transcript-debate-consultancy/Human-debate/debate_all_negative_belief_data_persuasion.csv", index=False)

In [ ]:
persuaded = df[df['veracity'] != df['answer_defending']] #incorrect
for item, _ in strategies.items():
    # print(persuaded.values[0])
    print(persuaded[f"{item}"].value_counts())
    print(len(persuaded))
    print('-'*50)

critical_thinking
3.0    41
3.5    30
2.5    27
2.0    22
4.0     5
1.5     4
1.0     2
Name: count, dtype: int64
131
--------------------------------------------------
alternative_explanations
3.0    44
3.5    26
2.5    24
2.0    13
1.0    12
1.5     7
4.0     5
Name: count, dtype: int64
131
--------------------------------------------------
harm
1.0    106
2.0      8
1.5      7
3.5      4
3.0      3
4.0      2
2.5      1
Name: count, dtype: int64
131
--------------------------------------------------
stories
1.0    101
1.5     24
2.0      3
2.5      2
3.0      1
Name: count, dtype: int64
131
--------------------------------------------------
conflicting_evidence
3.5    47
4.0    41
3.0    26
2.5     9
2.0     6
1.0     1
1.5     1
Name: count, dtype: int64
131
--------------------------------------------------
inconsistencies
1.5    32
1.0    31
2.0    24
3.0    22
2.5    12
3.5    10
Name: count, dtype: int64
131
--------------------------------------------------
